In [1]:
import os
from google.colab import output
from google.colab import files

file_selector = files.upload()

if not file_selector:
  print("No file selected.")

uploaded_file = next(iter(file_selector.values()))

try:
  if not uploaded_file.name.endswith(".xlsx"):
    print("Please select an .xlsx file.")
except:
  print("Please check file type")

with open("/content/data.xlsx", "wb") as f:
  f.write(uploaded_file)

# Print a success message
print("File uploaded and saved as data.xlsx.")



Saving data.xlsx to data.xlsx
Please check file type
File uploaded and saved as data.xlsx.


Verify that the excel file has been added to the current directory.

> Need to verify later if the pickle files of both @Sairam & @Swastik are added to the current directory

In [2]:
!mkdir "/content/Model/"
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
!mv "/content/model-bert-use-this.hdf5" "/content/Model/model-bert-use-this.hdf5"
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model_weights_and_optimizer.pickle
!mv "/content/model_weights_and_optimizer.pickle" "/content/Model/model_weights_and_optimizer.pickle"
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/random_forest_model.joblib
!mv "/content/random_forest_model.joblib" "/content/Model/random_forest_model.joblib"
!wget https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/vectorizer.joblib
!mv "/content/vectorizer.joblib" "/content/Model/vectorizer.joblib"

--2024-04-18 18:18:00--  https://github.com/ujwalnk/CIE-Candidate-Selection/raw/main/Models/model-bert-use-this.hdf5
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ujwalnk/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5 [following]
--2024-04-18 18:18:00--  https://raw.githubusercontent.com/ujwalnk/CIE-Candidate-Selection/main/Models/model-bert-use-this.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 418600 (409K) [application/octet-stream]
Saving to: ‘model-bert-use-this.hdf5’

model-bert-use-this 100%[===================>] 408.79K  --.-KB/s    in 0.04s   

2024-04-18 18:18:00

In [3]:
import pandas as pd


full_dataframe = pd.read_excel('/content/data.xlsx')

In [4]:
# Sairam 7 questions

questions = [
    "Give one/top reason why you want to take up this course",
    "What (one/top thing) do you expect to get out of this course?",
    "What does entrepreneurship mean to you?",
    "Can you give one example of any prototype you have built?",
    "Optional question: Are you excited to work on any ideas/opportunities as part of this course. If so please elaborate."
  ]

AI Component Detection Below

In [5]:
!pip install tensorflow_text
!pip install --upgrade tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [6]:
#run this cell once before predicting values (loading the model)
#Accuracy of the model = 82-86%
import tensorflow
import pickle
from tensorflow import keras
# try:
import tensorflow_hub as hub
import tensorflow_text as text  # Registers the ops.
# except:
#   import os
#   os.system("pip install tensorflow_text")
#   os.system("pip install --upgrade tf-keras")

#   import tensorflow_hub as hub
#   import tensorflow_text as text  # Registers the ops.

import numpy as np
import gzip

class BertEncoder(tensorflow.keras.layers.Layer):
    def __init__(self, preprocess_path, model_path, **kwargs):
        super(BertEncoder, self).__init__(**kwargs)
        self.preprocessor = hub.KerasLayer(preprocess_path)
        self.encoder = hub.KerasLayer(model_path, trainable=True)

    def call(self, inputs):
        encoder_inputs = self.preprocessor(inputs)
        outputs = self.encoder(encoder_inputs)
        pooled_output = outputs['pooled_output']  # [batch_size, 512].
        sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].
        return pooled_output, sequence_output

with tensorflow.keras.utils.custom_object_scope({'BertEncoder': BertEncoder}):
    # Load the model architecture and weights
    model = tensorflow.keras.models.load_model('/content/Model/model-bert-use-this.hdf5')

# Load model weights and optimizer state from pickle file
with open('/content/Model/model_weights_and_optimizer.pickle', 'rb') as f:
    weights, optimizer_config = pickle.load(f)

# Set model weights
model.set_weights(weights)

# Reconstruct optimizer from config
optimizer = tensorflow.keras.optimizers.Adam.from_config(optimizer_config)

print("Model Loaded")

Model Loaded


In [7]:
y_main=[]
for i in questions:
  temp1=[]
  y_pred=model.predict(full_dataframe[i].fillna("NA"))
  for j in y_pred:
    temp1.append(j[0])
  y_pred
  y_main.append(temp1)

7/7 ━━━━━━━━━━━━━━━━━━━━ 26s 4s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step


In [8]:
y_main = [list(i) for i in zip(*y_main)]#transposing the matrix
ai_df=pd.DataFrame(y_main)

In [9]:
ai_df # Contains the value of how probable the sentence was AI written

,0,1,2,3,4
0,0.436043,0.439359,0.612867,0.261363,0.255667
1,0.598018,0.524786,0.595584,0.505551,0.295410
2,0.574488,0.428131,0.705146,0.545257,0.385133
3,0.512077,0.578530,0.522148,0.447722,0.545257
4,0.481345,0.536223,0.336764,0.280010,0.545257
...,...,...,...,...,...
192,0.371537,0.509698,0.430067,0.401940,0.330762
193,0.795699,0.780257,0.559001,0.457133,0.277110
194,0.461140,0.382438,0.664772,0.420496,0.356717
195,0.577070,0.562858,0.573586,0.494900,0.440993


Natural Language Processing Below

In [10]:
# Your output df will be like this I presume, please use the same variable names.
import numpy as np
# Creating a random one to test it out
nlp_df = pd.DataFrame(np.random.rand(100, 1))

print(nlp_df)

import pandas as pd
from joblib import load

# Load the vectorizer
loaded_vectorizer = load('/content/Model/vectorizer.joblib')

# Load the model
loaded_model = load('/content/Model/random_forest_model.joblib')

# Read the data from data.xlsx
data_df = full_dataframe #pd.read_excel('/content/CIE L1_Aug2023 semester_OAT data.xlsx')

# Assuming the text responses are in columns 7 onwards
text_columns = data_df.columns[7:]

# Iterate over each row and make predictions
for index, row in data_df.iterrows():
    # Concatenate all text responses into a single string https://www.canva.com/design/DAGCG5bJlhs/ZaWlO8EVoVCsPdDpJqnM0A/edit?utm_content=DAGCG5bJlhs&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton
    answers_row = ' '.join(row[text_columns].astype(str))

    # Transform the text using the loaded vectorizer
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Make prediction for the row
    prediction = loaded_model.predict(answers_row_vec)

    # Print the predicted status
    predicted_status = 'confirmed' if prediction[0] == 1 else 'not confirmed'
    print(f"Row {index + 1} - Predicted status: {predicted_status}")

           0
0   0.740824
1   0.204566
2   0.299856
3   0.818424
4   0.961823
..       ...
95  0.821887
96  0.837617
97  0.881146
98  0.551604
99  0.737106

[100 rows x 1 columns]
Row 1 - Predicted status: not confirmed
Row 2 - Predicted status: not confirmed
Row 3 - Predicted status: not confirmed
Row 4 - Predicted status: not confirmed
Row 5 - Predicted status: not confirmed
Row 6 - Predicted status: not confirmed
Row 7 - Predicted status: not confirmed
Row 8 - Predicted status: confirmed
Row 9 - Predicted status: confirmed
Row 10 - Predicted status: not confirmed
Row 11 - Predicted status: confirmed
Row 12 - Predicted status: not confirmed
Row 13 - Predicted status: confirmed
Row 14 - Predicted status: not confirmed
Row 15 - Predicted status: not confirmed
Row 16 - Predicted status: not confirmed
Row 17 - Predicted status: not confirmed
Row 18 - Predicted status: not confirmed
Row 19 - Predicted status: not confirmed
Row 20 - Predicted status: confirmed
Row 21 - Predicted status: no

In [17]:
import pandas as pd
from joblib import load

# Load the vectorizer and model
loaded_vectorizer = load('/content/Model/vectorizer.joblib')
loaded_model = load('/content/Model/random_forest_model.joblib')

# Assume data_df is already defined and includes the required data
text_columns = data_df.columns[7:]

# List to store each row's data
data = []

# Iterate over rows in the dataframe
for index, row in full_dataframe.iterrows():
    # Vectorize the given row
    answers_row = ' '.join(row[text_columns].astype(str))
    answers_row_vec = loaded_vectorizer.transform([answers_row])

    # Predict the probability of the confirmed class
    probability = loaded_model.predict_proba(answers_row_vec)[:, 1]

    # Append dictionary to list
    data.append({'probability': float(probability[0])})

# Create DataFrame from list of dictionaries
results_df = pd.DataFrame(data)

# Optionally, to see the first few rows of your DataFrame:
print(results_df.head())

nlp_df = results_df

   probability
0     0.304626
1     0.398175
2     0.348375
3     0.349435
4     0.295705


Outputting to the Excel File

In [16]:
import pandas as pd
import numpy as np


# I am dividing it here to normalize the score

ai_df.loc[:, "AI Component"] = ai_df.sum(axis=1) / 5
ai_df = ai_df[["AI Component"]]

temp_df = full_dataframe
temp_df = pd.concat([temp_df, ai_df, nlp_df], axis=1)
temp_df.to_excel("/content/output.xlsx", index=True)

print("Output File Created!")

     AI Component
0        0.401060
1        0.503870
2        0.527631
3        0.521147
4        0.435920
..            ...
192      0.408801
193      0.573840
194      0.457113
195      0.529881
196      0.439204

[197 rows x 1 columns]


# Database

In [98]:
import sqlite3 as s
import datetime

def write_2_db(c, connection, param, value):
  c.execute("""
    INSERT INTO GRAPH_DATA (PARAM, YEAR, MONTH, VALUE) VALUES (?, ?, ?, ?)
  """, (param, datetime.date.today().year, datetime.date.today().month, value))

  # Save the transaction
  connection.commit()

connection = s.connect("/content/base.sqlite")

c = connection.cursor()

# Create a new table
c.execute("""
    CREATE TABLE IF NOT EXISTS GRAPH_DATA (
        PARAM TEXT NOT NULL,
        YEAR INT NOT NULL,
        MONTH INT NOT NULL,
        VALUE TEXT NOT NULL
    )
""")

In [99]:
write_2_db(c, connection, "AI", int(ai_df.mean()["AI Component"] * 100))

write_2_db(c, connection, "Branch/CS", int(full_dataframe["Branch"].value_counts()["CS"]))
write_2_db(c, connection, "Branch/EC", int(full_dataframe["Branch"].value_counts()["ECE"]))
write_2_db(c, connection, "Branch/EE", int(full_dataframe["Branch"].value_counts()["EEE"]))
write_2_db(c, connection, "Branch/AI", int(full_dataframe["Branch"].value_counts()["AIML"]))
write_2_db(c, connection, "Branch/BT", int(full_dataframe["Branch"].value_counts()["Biotech"]))
write_2_db(c, connection, "Branch/ME", int(full_dataframe["Branch"].value_counts()["Mech"]))

def get_year(x, y):
  sum = 0
  try:
    sum += int(full_dataframe["Semester"].value_counts()[x])
  except:
    try:
      sum += int(full_dataframe["Semester"].value_counts()[y])
    except:
      pass

  return sum

write_2_db(c, connection, "YEAR/1", int(get_year("1st","2nd")))
write_2_db(c, connection, "YEAR/2", int(get_year("3rd","4th")))
write_2_db(c, connection, "YEAR/3", int(get_year("5th","6th")))
write_2_db(c, connection, "YEAR/4", int(get_year("7th","8th")))

In [100]:
c.execute("SELECT * from GRAPH_DATA")
print(c.fetchall())

[('AI', 2024, 4, '48'), ('Branch/CS', 2024, 4, '79'), ('Branch/EC', 2024, 4, '50'), ('Branch/EE', 2024, 4, '22'), ('Branch/AI', 2024, 4, '22'), ('Branch/BT', 2024, 4, '16'), ('Branch/ME', 2024, 4, '8'), ('YEAR/1', 2024, 4, '0'), ('YEAR/2', 2024, 4, '139'), ('YEAR/3', 2024, 4, '56'), ('YEAR/4', 2024, 4, '2')]


<class 'pandas.core.series.Series'>


,2024 Apr,Branch,frequency
AI Component,48.997681,NaN,NaN
0,NaN,CS,79.0
1,NaN,ECE,50.0
2,NaN,EEE,22.0
3,NaN,AIML,22.0
